In [87]:
import torch 
import torch.nn as nn 
import torch.nn.functional as F 
from torch.utils.data import DataLoader
from torchsummary import summary

import cv2 
import os 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

from tqdm.auto import trange 

In [88]:
DATA = r"C:\Users\suyash\Desktop\indoml\train\train"
csv = r"C:\Users\suyash\Desktop\indoml\train_labels.csv"

In [107]:
BATCH=64
DATASIZE = 6400

# Preprocessing



In [147]:
datafile = pd.read_csv(csv).values
xtrain = []
ytrain = []
preprocessing = trange(DATASIZE, desc="Preprocessing Progress", unit="image", ncols=1000)
for i in preprocessing:
    y = datafile[i, 1]
    ytr = np.zeros(16, dtype="float32")
    ytr[y] = 1
    x = cv2.cvtColor(cv2.resize(cv2.imread(DATA+"/"+str(i)+".tif"), (225, 300), interpolation=cv2.INTER_AREA), cv2.COLOR_RGB2GRAY)
    x = (x/255)
    xtrain.append(x)
    ytrain.append(ytr)
    
X = np.array(xtrain, dtype="float32").reshape(-1, 300, 225, 1)
Y = np.array(ytrain, dtype="float32")

Preprocessing Progress:   0%|                                                                                 …

In [148]:
X.shape, Y.shape

((6400, 300, 225, 1), (6400, 16))

In [149]:
xtrain = torch.from_numpy(X)
ytrain = torch.from_numpy(Y)

xtrain = DataLoader(xtrain, batch_size=BATCH)
ytrain = DataLoader(ytrain, batch_size=BATCH)

xtrain = np.array([xtr for xtr in xtrain])
ytrain = np.array([ytr for ytr in ytrain])

C:\Users\suyash\AppData\Local\Temp/ipykernel_1272/3592386148.py:7: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  xtrain = np.array([xtr for xtr in xtrain])
C:\Users\suyash\AppData\Local\Temp/ipykernel_1272/3592386148.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  xtrain = np.array([xtr for xtr in xtrain])
C:\Users\suyash\AppData\Local\Temp/ipykernel_1272/359238

# Convolutional Neural Networks


In [150]:
class ResConvBlock(nn.Module):
    def __init__(self, filter, kernel=3):
        super(ResConvBlock, self).__init__()
        self.conv1 = nn.Conv2d(filter, filter, kernel)
        self.conv2 = nn.Conv2d(filter, filter, kernel)
        self.pad = nn.ZeroPad2d(1)
        self.pool = nn.MaxPool2d(2,2)
        self.norm1 = nn.BatchNorm2d(filter)
        self.norm2 = nn.BatchNorm2d(filter)
        self.lrelu = nn.LeakyReLU(0.1)

    def forward(self, x):
        s = x
        x = self.lrelu(self.norm1(self.pad(self.conv1(x))))
        x = self.lrelu(self.norm2(self.pad(self.conv2(x)))+s)
        return x

In [151]:
class ResBottleNeck(nn.Module):
    def __init__(self, infilter, outfilter, kernel=3):
        super(ResBottleNeck, self).__init__()
        self.conv0 = nn.Conv2d(infilter, infilter, 1)
        self.conv2 = nn.Conv2d(infilter, outfilter, 1)
        self.conv1 = nn.Conv2d(infilter, infilter, kernel, stride=2)
        self.conv = nn.Conv2d(infilter, outfilter, kernel, stride=2)
        self.pad = nn.ZeroPad2d(1)
        self.norm0 = nn.BatchNorm2d(infilter)
        self.norm1 = nn.BatchNorm2d(infilter)
        self.norm2 = nn.BatchNorm2d(outfilter)
        self.norm = nn.BatchNorm2d(outfilter)
        self.lrelu = nn.LeakyReLU(0.1)
        # self.pool = nn.MaxPool2d(2,2)

    def forward(self, x):
        s = x
        x = self.lrelu(self.norm0(self.conv0(x)))
        x = self.lrelu(self.norm1(self.pad(self.conv1(x))))
        x = self.lrelu(self.norm2(self.conv2(x)))
        s = self.lrelu(self.norm(self.pad(self.conv(s))))
        x = x+s
        return x

In [152]:
class ResBlock(nn.Module):
    def __init__(self, infilter, outfilter, kernel=3):
        super(ResBlock, self).__init__()
        self.conv0 = ResConvBlock(infilter)
        self.conv1 = ResConvBlock(infilter)
        self.conv2 = ResConvBlock(infilter)
        self.conv3 = ResConvBlock(infilter)

        self.conv = ResBottleNeck(infilter, outfilter)

    def forward(self, x):
        return self.conv(self.conv3(self.conv2((self.conv1(self.conv0(x))))))

In [153]:
class StartBlock(nn.Module):
    def __init__(self, filter):
        super(StartBlock, self).__init__()
        self.conv1 = nn.Conv2d(1, filter, 7, stride=2)
        # self.conv2 = nn.Conv2d(filter//2, filter, 5, stride=2)

        self.norm1 = nn.BatchNorm2d(filter)
        # self.norm2 = nn.BatchNorm2d(filter)

        self.lrelu = nn.LeakyReLU(0.1)

    def forward(self, x):
        x = self.lrelu(self.norm1(self.conv1(x)))
        return x #self.lrelu(self.norm2(self.conv2(x)))

In [154]:
class GMaxpool(nn.Module):
    def forward(self, x):
        return F.avg_pool2d(x, kernel_size=x.size()[2:])

In [155]:
class ResNet(nn.Module):
    def __init__(self, filter=16, ndim=256, outclass=16):
        super(ResNet, self).__init__()
        self.res0 = StartBlock(filter)
        self.res1 = ResBlock(filter, filter*2)
        self.res2 = ResBlock(filter*2, filter*4)
        self.res3 = ResBlock(filter*4, filter*8)
        self.res4 = ResBlock(filter*8, filter*16)
        self.res5 = ResBlock(filter*16, filter*32)
        self.res6 = ResBlock(filter*32, filter*64)

        self.avgpool = GMaxpool()
        self.flat = nn.Flatten()

        self.dense1 = nn.Linear(1024, ndim)
        self.dense2 = nn.Linear(ndim, outclass)
        self.drop = nn.Dropout2d(0.2)
        self.lrelu = nn.LeakyReLU(0.2)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.res6(self.res5(self.res4(self.res3(self.res2(self.res1(self.res0(x)))))))
        x = self.flat(self.avgpool(x))
        return self.softmax(self.dense2(self.lrelu(self.drop(self.dense1(x)))))

# Training 

#### labels --> one-hot 
one_hot = torch.nn.functional.one_hot(target)
#### one-hot --> labels
labels_again = torch.argmax(one_hot, dim=1)


In [156]:
model = ResNet()
summary(model, (1, 300,225))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 147, 110]             800
       BatchNorm2d-2         [-1, 16, 147, 110]              32
         LeakyReLU-3         [-1, 16, 147, 110]               0
        StartBlock-4         [-1, 16, 147, 110]               0
            Conv2d-5         [-1, 16, 145, 108]           2,320
         ZeroPad2d-6         [-1, 16, 147, 110]               0
       BatchNorm2d-7         [-1, 16, 147, 110]              32
         LeakyReLU-8         [-1, 16, 147, 110]               0
            Conv2d-9         [-1, 16, 145, 108]           2,320
        ZeroPad2d-10         [-1, 16, 147, 110]               0
      BatchNorm2d-11         [-1, 16, 147, 110]              32
        LeakyReLU-12         [-1, 16, 147, 110]               0
     ResConvBlock-13         [-1, 16, 147, 110]               0
           Conv2d-14         [-1, 16, 1

In [157]:
learning_rate = 1e-4
decay = 1
epochs = 1
criterion = nn.CrossEntropyLoss()

`Categorical Cross ENtropy`</br>
Input: (N,C) where C = number of classes
Target: (N) where each value is 0 <= targets[i] <= C-1
Output: scalar. If reduce is False, then (N) instead.

In [158]:
steps = len(xtrain)
for epoch in range(epochs):
    lss = 0
    learning_rate = learning_rate/(epoch*decay+1)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    print(f"Epoch {epoch+1}/{epochs}")
    train = trange(steps, desc=f"\tTraining: 0/{steps} steps || Loss: NaNaN || Step Loss: NaNaN || Progress", unit="steps", ncols=1000)
    for c in train:
        xtr = xtrain[c]
        ytr = ytrain[c]
        ypred = model(xtr.reshape(BATCH, 1, 300, 225))
        loss = criterion(ypred.reshape(-1,16), torch.argmax(ytr.reshape(-1,16), dim=1))
        lss = lss + loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train.set_description(f"\tTraining: {c+1}/{steps} steps || Loss: {lss/(c+1):.4f} || Step Loss: {loss:.4f} || Progress")


Epoch 1/1


	Training: 0/100 steps || Loss: NaNaN || Step Loss: NaNaN || Progress:   0%|                                  …

KeyboardInterrupt: 